In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV



In [ ]:
df = pd.read_csv('data/result.csv')

df.columns

Index(['is_home_win', 'home_form_5', 'away_form_5', 'home_form_10',
       'away_form_10', 'home_form_15', 'away_form_15',
       'last_home_matches_count', 'last_away_matches_count', 'days_off_home',
       'days_off_away', 'team_abbreviation_home', 'team_abbreviation_away',
       'is_last_season_champion_home', 'is_last_season_champion_away',
       'is_regular_season', 'is_playoffs', 'match_number_season_home',
       'match_number_season_away', 'home_over_away_wins_diff_5',
       'home_over_away_wins_diff_10'],
      dtype='object')

In [ ]:
df = df[['is_home_win', 'home_form_5', 'away_form_5', 'home_form_15', 'away_form_15', 'last_home_matches_count', 'last_away_matches_count', 'days_off_home', 'days_off_away', 'is_last_season_champion_home', 'is_last_season_champion_away', 'is_playoffs', 'home_over_away_wins_diff_10']]

In [ ]:
df.columns

Index(['is_home_win', 'home_form_5', 'away_form_5', 'home_form_15',
       'away_form_15', 'last_home_matches_count', 'last_away_matches_count',
       'days_off_home', 'days_off_away', 'is_last_season_champion_home',
       'is_last_season_champion_away', 'is_playoffs',
       'home_over_away_wins_diff_10'],
      dtype='object')

In [ ]:
df.describe()

,is_home_win,home_form_5,away_form_5,home_form_15,away_form_15,last_home_matches_count,last_away_matches_count,days_off_home,days_off_away,is_last_season_champion_home,is_last_season_champion_away,is_playoffs,home_over_away_wins_diff_10
count,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000,13720.000000
mean,0.583382,2.552405,2.556633,7.445773,7.441181,1.024052,1.003790,2.517784,2.495700,0.038630,0.037391,0.072668,-0.020554
std,0.493016,1.138564,1.129749,2.178988,2.156237,1.475560,1.412558,1.465189,1.428617,0.192718,0.189724,0.259600,2.583149
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-10.000000
25%,0.000000,2.000000,2.000000,6.000000,6.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,-2.000000
50%,1.000000,3.000000,3.000000,8.000000,8.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,3.000000,3.000000,9.000000,9.000000,2.000000,2.000000,3.000000,3.000000,0.000000,0.000000,0.000000,2.000000
max,1.000000,5.000000,5.000000,14.000000,14.000000,12.000000,11.000000,16.000000,15.000000,1.000000,1.000000,1.000000,10.000000


In [ ]:
df['form_5_diff'] = df['home_form_5'] - df['away_form_5']
df['form_15_diff'] = df['home_form_15'] - df['away_form_15']
df['last_home_matches_count_diff'] = df['last_home_matches_count'] - df['last_away_matches_count']
df['days_off_diff'] = df['days_off_home'] - df['days_off_away']
df['is_last_season_champion_diff'] = df['is_last_season_champion_home'] - df['is_last_season_champion_away']
df = df[df['is_playoffs'] == 0].reset_index(drop=True)

df.drop(columns=['home_form_5', 'away_form_5', 'home_form_15', 'away_form_15', 'last_home_matches_count', 'last_away_matches_count', 'days_off_home', 'days_off_away', 'is_last_season_champion_home', 'is_last_season_champion_away', 'is_playoffs'], inplace=True)

In [ ]:
print(df.columns)
display(df.describe())

Index(['is_home_win', 'home_over_away_wins_diff_10', 'form_5_diff',
       'form_15_diff', 'last_home_matches_count_diff', 'days_off_diff',
       'is_last_season_champion_diff'],
      dtype='object')


,is_home_win,home_over_away_wins_diff_10,form_5_diff,form_15_diff,last_home_matches_count_diff,days_off_diff,is_last_season_champion_diff
count,12723.000000,12723.000000,12723.000000,12723.000000,12723.000000,12723.000000,12723.000000
mean,0.580838,-0.021221,-0.007938,0.003851,0.018706,0.024130,-0.000236
std,0.493441,2.578225,1.610232,2.796072,2.019469,1.785626,0.260452
min,0.000000,-10.000000,-5.000000,-11.000000,-11.000000,-13.000000,-1.000000
25%,0.000000,-2.000000,-1.000000,-2.000000,-1.000000,-1.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000
max,1.000000,10.000000,5.000000,11.000000,11.000000,10.000000,1.000000


Что сделали к этому этапу? Оставили поменьше признаков, чтобы модель не переобучалась + сделали существующие признаки разностными, исследования показывают, что в таком виде моделям удобнее их воспринимать (имеется в виду, вместо двух колонок home_form и away_form мы делаем одну form_diff)

In [ ]:
df = df.tail(5000).reset_index(drop=True)


In [ ]:
df.head()

,is_home_win,home_over_away_wins_diff_10,form_5_diff,form_15_diff,last_home_matches_count_diff,days_off_diff,is_last_season_champion_diff
0,1,2.0,2.0,4.0,1.0,0.0,0.0
1,0,2.0,-1.0,-1.0,0.0,-2.0,0.0
2,1,-8.0,2.0,1.0,-2.0,0.0,0.0
3,1,-2.0,1.0,-1.0,1.0,0.0,1.0
4,1,-4.0,0.0,-1.0,0.0,-2.0,0.0


In [ ]:
# Сгруппируем данные по уникальным наборам признаков, посчитаем количество побед для домашей и гостевой команды для каждого случая

grouped = df.groupby(list(X.columns)).agg(
    away_win_count = ('is_home_win', lambda x: (x == 0).sum()),
    home_win_count = ('is_home_win', lambda x: (x == 1).sum())
).reset_index()
grouped.shape
# Выведем топ строк по количеству домашних побед (home_win_count) в grouped
print("Топ 10 строк по количеству домашних побед:")
display(grouped.sort_values(by='home_win_count', ascending=False).head(10))

print("Топ 10 строк по количеству гостевых побед:")
display(grouped.sort_values(by='away_win_count', ascending=False).head(10))


Топ 10 строк по количеству домашних побед:


,home_over_away_wins_diff_10,form_5_diff,form_15_diff,last_home_matches_count_diff,days_off_diff,is_last_season_champion_diff,away_win_count,home_win_count
2672,2.0,-1.0,-1.0,0.0,0.0,0.0,0,7
1800,0.0,-1.0,-1.0,0.0,0.0,0.0,6,6
2778,2.0,0.0,-2.0,0.0,0.0,0.0,2,6
1186,-2.0,0.0,2.0,0.0,0.0,0.0,2,5
1964,0.0,0.0,-1.0,-1.0,0.0,0.0,2,5
598,-4.0,1.0,0.0,0.0,0.0,0.0,1,5
987,-2.0,-1.0,-1.0,0.0,0.0,0.0,1,4
1280,-2.0,1.0,0.0,0.0,0.0,0.0,3,4
418,-4.0,-1.0,-1.0,0.0,0.0,0.0,1,4
3097,2.0,2.0,1.0,0.0,0.0,0.0,1,4


Топ 10 строк по количеству гостевых побед:


,home_over_away_wins_diff_10,form_5_diff,form_15_diff,last_home_matches_count_diff,days_off_diff,is_last_season_champion_diff,away_win_count,home_win_count
1800,0.0,-1.0,-1.0,0.0,0.0,0.0,6,6
2823,2.0,0.0,0.0,0.0,0.0,0.0,5,2
1968,0.0,0.0,-1.0,0.0,0.0,0.0,5,3
2574,2.0,-2.0,-1.0,0.0,0.0,0.0,4,0
970,-2.0,-1.0,-2.0,1.0,0.0,0.0,4,0
1349,-2.0,1.0,3.0,0.0,0.0,0.0,4,1
3485,4.0,0.0,1.0,0.0,0.0,0.0,3,3
3295,4.0,-2.0,-1.0,0.0,1.0,0.0,3,0
321,-4.0,-2.0,-4.0,0.0,0.0,0.0,3,1
1824,0.0,-1.0,0.0,0.0,0.0,0.0,3,2


In [ ]:
# Если домашняя команда при прочих равных побеждает чаще или столько же, сколько и гостевая, то считаем, что домашняя команда победит

grouped['is_home_win'] = grouped['home_win_count'] >= grouped['away_win_count']
grouped.drop(columns=['home_win_count', 'away_win_count'], inplace=True)
grouped['is_home_win'] = grouped['is_home_win'].astype(int)
display(grouped.head(20))
grouped.shape


,home_over_away_wins_diff_10,form_5_diff,form_15_diff,last_home_matches_count_diff,days_off_diff,is_last_season_champion_diff,is_home_win
0,-10.0,-2.0,-2.0,-4.0,-3.0,0.0,0
1,-10.0,-1.0,-4.0,3.0,0.0,0.0,0
2,-10.0,-1.0,-2.0,-3.0,0.0,0.0,1
3,-10.0,-1.0,-2.0,3.0,1.0,0.0,0
4,-10.0,-1.0,0.0,1.0,0.0,0.0,1
5,-10.0,0.0,0.0,2.0,0.0,0.0,1
6,-10.0,0.0,3.0,-1.0,-2.0,0.0,0
7,-10.0,1.0,0.0,-4.0,2.0,0.0,0
8,-10.0,1.0,0.0,-2.0,0.0,0.0,0
9,-8.0,-4.0,-4.0,-2.0,0.0,0.0,1


(4007, 7)

In [ ]:
grouped.head()

,home_over_away_wins_diff_10,form_5_diff,form_15_diff,last_home_matches_count_diff,days_off_diff,is_last_season_champion_diff,is_home_win
0,-10.0,-2.0,-2.0,-4.0,-3.0,0.0,0
1,-10.0,-1.0,-4.0,3.0,0.0,0.0,0
2,-10.0,-1.0,-2.0,-3.0,0.0,0.0,1
3,-10.0,-1.0,-2.0,3.0,1.0,0.0,0
4,-10.0,-1.0,0.0,1.0,0.0,0.0,1


In [ ]:
X = grouped.drop(columns=['is_home_win'])
y = grouped['is_home_win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(3205, 6) (802, 6) (3205,) (802,)


In [ ]:
# Посчитаем количество дубликатов в данных
num_duplicates = X.duplicated().sum()
print(f"Количество дубликатов в данных: {num_duplicates}")


Количество дубликатов в данных: 0


In [ ]:
# Поскейлим признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)


In [ ]:
# Посмотрим распределение данных в ответе
print(y_train.value_counts(normalize=True)[1] * 100)
print(y_test.value_counts(normalize=True)[1] * 100)


60.280811232449295
60.349127182044896


In [ ]:
# Попробуем обычную логистическую регрессию

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

# Получим веса (коэффициенты) модели после обучения
print("Коэффициенты (weights) модели:")
for feature, weight in zip(X_train.columns, lr.coef_[0]):
    print(f"{feature}: {weight:.4f}")

print("Смещение (intercept):", lr.intercept_[0])


0.6034912718204489
0.5
0.7527216174183515
Коэффициенты (weights) модели:
home_over_away_wins_diff_10: 0.0221
form_5_diff: -0.0077
form_15_diff: 0.0055
last_home_matches_count_diff: 0.0196
days_off_diff: 0.0095
is_last_season_champion_diff: 0.0129
Смещение (intercept): 0.4172999860266491


In [ ]:
# Попробуем сделать классификацию с помощью логистической регрессии, сразу найдем лучший гиперпараметр С (используя метрику roc_auc)

params = {'C' : [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]} # диапазон подобрал экспериментально, от 0.001 до 10000 это лучше

gs = GridSearchCV(LogisticRegression(), params, cv=3, scoring='roc_auc')
gs.fit(X, y)

print(gs.best_score_)
print(gs.best_params_)

# Найдем значение accuracy, полученное от такого С

lr = LogisticRegression(C=gs.best_params_['C'])
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print('accuracy', accuracy_score(y_test, y_pred)) # Просто для информации, чтобы знать, насколько хорошо работает модель

print('-' * 50)

# Теперь то же самое для f1

params = {'C' : [1e-15, 1e-14, 1e-13, 1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]} # диапазон подобрал экспериментально

gs = GridSearchCV(LogisticRegression(), params, cv=3, scoring='f1')
gs.fit(X, y)

print(gs.best_score_)
print(gs.best_params_)

# Найдем значение accuracy, полученное от такого С

lr = LogisticRegression(C=gs.best_params_['C'])
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print('accuracy', accuracy_score(y_test, y_pred)) # Просто для информации, чтобы знать, насколько хорошо работает модель

print('-' * 50)

# Теперь попробуем SVM

svm = SVC()
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print('accuracy', accuracy_score(y_test, y_pred))
print('roc_auc', roc_auc_score(y_test, y_pred))
print('f1', f1_score(y_test, y_pred))



0.476383270666133
{'C': 70}
accuracy 0.6034912718204489
--------------------------------------------------
0.7522963986851711
{'C': 1e-10}
accuracy 0.6034912718204489
--------------------------------------------------
accuracy 0.5985037406483791
roc_auc 0.4974855761733977
f1 0.7476489028213166


In [ ]:
# Теперь попробуем SVM с разными ядрами

kernels = ['linear', 'rbf', 'poly', 'sigmoid']

for kernel in kernels:
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)

    y_pred = svm.predict(X_test)
    print(f'Ядро: {kernel}')
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')
    print(f'F1: {f1_score(y_test, y_pred)}')
    print('-' * 50)

# rbf лучше всего работает, хоть и все еще недостаточно хорошо, попробуем подобрать гиперпараметры

# params = {'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# gs = GridSearchCV(SVC(kernel='rbf'), params, cv=3, scoring='roc_auc')
# gs.fit(X, y)

# print(gs.best_score_)
# print(gs.best_params_)

Ядро: linear
Accuracy: 0.6034912718204489
ROC AUC: 0.5
F1: 0.7527216174183515
--------------------------------------------------
Ядро: rbf
Accuracy: 0.5985037406483791
ROC AUC: 0.4974855761733977
F1: 0.7476489028213166
--------------------------------------------------
Ядро: poly
Accuracy: 0.5997506234413965
ROC AUC: 0.496900826446281
F1: 0.7498051441932969
--------------------------------------------------
Ядро: sigmoid
Accuracy: 0.5149625935162094
ROC AUC: 0.49621861843131143
F1: 0.5935214211076281
--------------------------------------------------
